# Import Libraries

In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.models import Sequential
import pickle
import warnings

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/alper/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Import Dataset

In [7]:
df = pd.read_csv("sqli.csv", encoding="utf-16")

In [8]:
df.head()

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1


In [11]:
df.count()

Sentence    4187
Label       4200
dtype: int64

# Vectorization

In [12]:
max_df = 0.7
max_features = 4096

In [13]:
stopwords_lst = stopwords.words("english")

In [16]:
stopwords_lst

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [17]:
vector = CountVectorizer(
    max_df=max_df,
    max_features=max_features,
    stop_words=stopwords_lst
)

# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
    # "Convert a collection of text documents to a matrix of token counts."

# https://www.geeksforgeeks.com/using-countrvectorizer-to-extracting-features-from-text/
    # It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.
    # This is helpful when we have multiple such texts, and we wish to convert each word in each text into vectors (for using in further text analysis).

In [18]:
vector

CountVectorizer(max_df=0.7, max_features=4096,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [19]:
type(vector)

sklearn.feature_extraction.text.CountVectorizer

In [21]:
vector.max_df
vector.max_features
vector.stop_words

4096

In [22]:
data_val = df["Sentence"]

In [23]:
data_val

0                                                       a
1                                                     a' 
2                                                   a' --
3                                         a' or 1 = 1; --
4                                                       @
                              ...                        
4195     org/?option = com_k2  <a href = "http://corfopym
4196    com/?option = com_k2  <act>  <![CDATA[procMemb...
4197                                            picsearch
4198    com/is?-WZx-uhyLezKNiYLvAbKL3W4oh5F749nr2KUmFF...
4199    de]]>  </email_address>   <find_account_answer...
Name: Sentence, Length: 4200, dtype: object

In [24]:
data_val[30:40]

30          ' or username is not NULL or username  =  '
31    1 and ascii ( lower ( substring  (  (  select ...
32    1 union all select 1,2,3,4,5,6,name from sysob...
33                    1 uni/**/on select all from where
34                                        ’ or ‘1’ = ’1
35                                        ' or '1' = '1
36    '||utl_http.request ( 'httP://192.168.1.1/' ) ||'
37    ' || myappadmin.adduser ( 'admin', 'newpass' )...
38    ' AND 1 = utl_inaddr.get_host_address  (  (  S...
39    ' AND 1 = utl_inaddr.get_host_address  (  (  S...
Name: Sentence, dtype: object

In [25]:
data_val[100:150]

100                                         ' or 1 = 1--
101                                         " or 1 = 1--
102                                        ' or 1 = 1 /*
103                                           or 1 = 1--
104                                        ' or 'a' = 'a
105                                        " or "a" = "a
106                                  ' )  or  ( 'a' = 'a
107                                          admin' or '
108          ' select * from information_schema.tables--
109     )  union select * from information_schema.tab...
110                                     ' having 1 = 1--
111                                     ' having 1 = 1--
112                     ' group by userid having 1 = 1--
113    ' select name from syscolumns where id  =   ( ...
114                   ' or 1 in  ( select @@version ) --
115                       ' union all select @@version--
116                         ' or 'unusual'  =  'unusual'
117                  ' or 'some

In [26]:
data_val2 = df["Sentence"].values

In [27]:
data_val2

array(['a', "a' ", "a' --", ..., 'picsearch',
       'com/is?-WZx-uhyLezKNiYLvAbKL3W4oh5F749nr2KUmFFjZOU  <email_address>  <![CDATA[elwoodbuckley@freenet',
       'de]]>  </email_address>   <find_account_answer>  <![CDATA[fqaepqdkct]]>  </find_account_answer>   <find_account_question>  <![CDATA[6]]>  </find_account_question>   <homepage>  <![CDATA[]]>  </homepage>   <is_secret>  <![CDATA[]]>  </is_secret>   <methodCall>   <mid>  <![CDATA[qna]]>  </mid>   <module>  <![CDATA[member]]>  </module>   <nick_name>  <![CDATA[ElwoodBuckley041104]]>  </nick_name>   <params>   <password2>  <![CDATA[Yxg6dsF5iU]]>  </password2>   <password>  <![CDATA[Yxg6dsF5iU]]>  </password>   <reward_point>  <![CDATA[0]]>  </reward_point>   <title>  <![CDATA[Ways To Make Cash Online - Make Money From Home]]>  </title>   <user_id>  <![CDATA[ElwoodBuckley041104]]>  </user_id>   <user_name>  <![CDATA[ElwoodBuckley041104]]>  </user_name>   =  =   =  = 01 @7520 A3B10 A3B12C41 A3B12C411 APP AR-203 ASCd AgAHMzM3MsdfjMw

In [28]:
len(data_val2)

4200

In [29]:
data_lbl = df["Label"]

In [30]:
data_lbl

0       1
1       1
2       1
3       1
4       1
       ..
4195    0
4196    0
4197    0
4198    0
4199    0
Name: Label, Length: 4200, dtype: int64

In [32]:
data_lbl[30:40]

30    1
31    1
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    1
Name: Label, dtype: int64

In [33]:
data_lbl[100:150]

100    1
101    1
102    1
103    1
104    1
105    1
106    1
107    1
108    1
109    1
110    1
111    1
112    1
113    1
114    1
115    1
116    1
117    1
118    1
119    1
120    1
121    1
122    1
123    1
124    1
125    1
126    1
127    1
128    1
129    1
130    1
131    1
132    1
133    1
134    1
135    1
136    1
137    1
138    1
139    1
140    1
141    1
142    1
143    1
144    1
145    1
146    1
147    1
148    1
149    1
Name: Label, dtype: int64

In [34]:
data_lbl2 = df["Label"].values

In [35]:
data_lbl2

array([1, 1, 1, ..., 0, 0, 0])

In [36]:
len(data_lbl2)

4200

In [37]:
n_sent = vector.fit_transform(df["Sentence"].values.astype("U")).toarray()

# fit_transform : Fit to data, then transform it.
    # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler.fit_transform

In [38]:
n_sent

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 2]])

In [39]:
len(n_sent)

4200

In [40]:
print("Sentences: ", vector.vocabulary_)
# Identified Unique Words
# Indices

Sentences:  {'union': 3829, 'userid': 3866, 'null': 2491, 'email': 1248, 'anything': 309, 'select': 3271, 'count': 966, 'tabname': 3633, 'members': 2308, 'full_name': 1501, 'like': 2126, 'bob': 537, '23': 55, 'exec': 1320, 'master': 2265, 'xp_cmdshell': 4061, 'ping': 2695, '172': 26, '10': 4, '255': 59, 'user_name': 3865, 'dbo': 1053, 'desc': 1103, 'users': 3868, 'username': 3867, 'lower': 2191, 'substring': 3573, 'top': 3729, 'name': 2436, 'sysobjects': 3625, 'xtype': 4063, '116': 17, 'uni': 3828, 'utl_http': 3875, 'request': 3039, 'http': 1740, 'admin': 184, 'utl_inaddr': 3876, 'get_host_address': 1534, 'banner': 448, 'version': 3908, 'rownum': 3144, 'sys': 3624, 'login_user': 2163, 'dual': 1194, 'instance': 1858, 'distinct': 1157, 'all_users': 240, 'password': 2631, 'user': 3863, 'table_name': 3629, 'all_tables': 239, 'column_name': 799, 'all_tab_columns': 238, 'granted_role': 1571, 'dba_role_privs': 1052, 'grantee': 1572, 'limit': 2129, 'loginuser': 2164, 'bin': 511, '15': 22, 'hex

In [41]:
vector.vocabulary_.keys()

dict_keys(['union', 'userid', 'null', 'email', 'anything', 'select', 'count', 'tabname', 'members', 'full_name', 'like', 'bob', '23', 'exec', 'master', 'xp_cmdshell', 'ping', '172', '10', '255', 'user_name', 'dbo', 'desc', 'users', 'username', 'lower', 'substring', 'top', 'name', 'sysobjects', 'xtype', '116', 'uni', 'utl_http', 'request', 'http', 'admin', 'utl_inaddr', 'get_host_address', 'banner', 'version', 'rownum', 'sys', 'login_user', 'dual', 'instance', 'distinct', 'all_users', 'password', 'user', 'table_name', 'all_tables', 'column_name', 'all_tab_columns', 'granted_role', 'dba_role_privs', 'grantee', 'limit', 'loginuser', 'bin', '15', 'hex', 'char', '45', 'isnull', '7659', 'information_schema', 'tables', 'group', 'id', 'unusual', 'something', 'thing', 'text', 'whatever', '37', 'login', '111', 'sel', 'ect', 'execute', 'immediate', 'us', 'er', 'begin', 'declare', 'var', 'varchar', '8000', 'set', 'temp', 'end', 'load_file', 'etc', 'passwd', '47', '101', '99', '112', '115', '100', 

In [42]:
len(vector.vocabulary_.keys())

4096

In [43]:
print(n_sent)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 2]]


In [44]:
n_sent.shape = (4200, 64, 64, 1)

In [45]:
n_sent.shape

(4200, 64, 64, 1)

In [46]:
X = n_sent
y = df["Label"]

In [48]:
print(X)
print(y)

[[[[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  ...

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]]


 [[[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  ...

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]]


 [[[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  ...

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]]


 ...


 [[[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]

In [49]:
# vector.get_feature_names_out()
# len(vector.get_feature_names_out())
# vector.get_feature_names_out()[100]

# Get output feature names for transformation.
    # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

array(['00', '000', '01', ..., 'zoomlion', 'zweig', '트리거'], dtype=object)

# Dataset (Training & Testing Split)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
trainX = X_train.copy()
trainX.shape = (trainX.shape[0], trainX.shape[1] * trainX.shape[2])

testX = X_test.copy()
testX.shape = (testX.shape[0], testX.shape[1] * testX.shape[2])

In [53]:
len(trainX)

3360

In [54]:
len(testX)

840

# Neural Network (Convolutional Neural Network)

In [59]:
model = Sequential()

In [60]:
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu", input_shape=(64, 64, 1))) # Input Layer
model.add(tf.keras.layers.MaxPooling2D((2, 2))) # Pooling Layer
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu")) # Convolution Layer
model.add(tf.keras.layers.MaxPooling2D((2, 2))) # Pooling Layer
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation="relu")) # Hidden layer 1
model.add(tf.keras.layers.Dense(128, activation="relu")) # Hidden layer 2
model.add(tf.keras.layers.Dense(1, activation="sigmoid")) # Output Layer

In [61]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [62]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 128)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 64)               

In [64]:
classifier_nn = model.fit(
    X_train,
    y_train,
    epochs = 10,
    validation_data = (X_test, y_test),
    batch_size = 32
)

Epoch 1/10


2023-10-18 19:53:30.513027: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8904
2023-10-18 19:53:30.780764: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-18 19:53:31.367195: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-18 19:53:31.931089: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc7b40ba910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-18 19:53:31.931115: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-10-18 19:53:31.943056: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-18 19:53:32.074848: I ./tensorflow/compiler/jit/device_comp

105/105 [==============================] - 6s 15ms/step - loss: 0.1598 - accuracy: 0.9262 - val_loss: 0.1146 - val_accuracy: 0.9536
Epoch 2/10
105/105 [==============================] - 1s 11ms/step - loss: 0.0853 - accuracy: 0.9640 - val_loss: 0.0896 - val_accuracy: 0.9583
Epoch 3/10
105/105 [==============================] - 1s 11ms/step - loss: 0.0788 - accuracy: 0.9673 - val_loss: 0.0966 - val_accuracy: 0.9583
Epoch 4/10
105/105 [==============================] - 1s 12ms/step - loss: 0.0647 - accuracy: 0.9729 - val_loss: 0.0967 - val_accuracy: 0.9655
Epoch 5/10
105/105 [==============================] - 1s 12ms/step - loss: 0.0566 - accuracy: 0.9777 - val_loss: 0.0942 - val_accuracy: 0.9714
Epoch 6/10
105/105 [==============================] - 1s 12ms/step - loss: 0.0543 - accuracy: 0.9777 - val_loss: 0.1185 - val_accuracy: 0.9643
Epoch 7/10
105/105 [==============================] - 1s 12ms/step - loss: 0.0556 - accuracy: 0.9780 - val_loss: 0.0897 - val_accuracy: 0.9702
Epoch 8/10

# Model Prediction

In [65]:
prediction = model.predict(X_test)

27/27 [==============================] - 0s 3ms/step


In [67]:
for i in range(len(prediction)):
    if prediction[i] > 0.5:
        prediction[i] = 1
    elif prediction[i] <= 0.5:
        prediction[i] = 0        

In [68]:
model.save("sqli_model_cnn.keras")

In [71]:
with open("vect_cnn_2", "wb") as f:
    pickle.dump(vector, f)

In [72]:
def accuracy_function(tp, tn, fp, fn):
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return accuracy

In [79]:
def precision_function(tp, fp):
    precision = tp / (tp + fp)
    return precision

In [80]:
def recall_function(tp, fn):
    recall = tp / (tp + fn)
    return recall

# Confusion Matrix

In [81]:
def confusion_matrix(actual, predicted):
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0

    for true, pred in zip(actual, predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1

    accuracy = accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision = precision_function(true_positive, false_positive)
    recall = recall_function(true_positive, false_negative)

    return (accuracy, precision, recall)

# Accuracy & Precision & Recall

In [82]:
accuracy, precision, recall = confusion_matrix(y_test, prediction)

In [83]:
print(" Accuracy Score: {0}\n Precision : {1}\n Recall: {2}".format(accuracy, precision, recall))

 Accuracy Score: 0.969047619047619
 Precision : 0.9094202898550725
 Recall: 0.996031746031746


# Model Testing

In [84]:
modelx = tf.keras.models.load_model("sqli_model_cnn.keras")

In [85]:
vectorizerx = pickle.load(open("vect_cnn_2", "rb"))

In [91]:
def predict_sqlinjection():
    repeat = True

    input_val = input("SQL Query : ")

    if input_val == '0':
        repeat = False

    input_val = [input_val]
    input_val = vectorizerx.transform(input_val).toarray()
    input_val.shape = (1, 64, 64, 1)

    result = modelx.predict(input_val)

    if repeat == True:
        if result > 0.5:
            print("SQL Injection Attack Detected!")
        elif result <= 0.5:
            print("Safe SQL query.")
        predict_sqlinjection()
    elif repeat == False:
        print("finish.")

In [92]:
predict_sqlinjection()

SQL Query :  SELECT * FROM email = 'cihan@deeplab.co' OR 1=1;


1/1 [==============================] - 0s 144ms/step
SQL Injection Attack Detected!


SQL Query :  0


1/1 [==============================] - 0s 23ms/step
finish.
